# Gas Price Recommendation

This notebook evaluate the approach on a simulation. It runs a simulation for the 'Geth' strategy too, as a baseline candidate.

In [1]:
# Install the `deep_gas_oracle` package !
!pip install ../

import os
from pathlib import Path
import numpy as np
import pandas as pd
from google.cloud import bigquery
import torch
import matplotlib.pyplot as plt

from deep_gas_oracle.preprocessing.fetch import fetch_eth_blocks
from deep_gas_oracle.preprocessing.norm import Normalizer
from deep_gas_oracle.modeling.model import GruMultiStep
from deep_gas_oracle.recommendation.oracle import EthGasPriceOracle
from deep_gas_oracle.recommendation.baseline import geth_recommend
from deep_gas_oracle.recommendation.benchmark import evaluate_recommender
# TODO: plot the final price recommendation versus the mean min and max

Processing /home/luisao/perso/deep_gas_oracle


  Created wheel for deep-gas-oracle: filename=deep_gas_oracle-0.1-py3-none-any.whl size=13466 sha256=a4cbdbf05d5244b761815ea5ab32124e7acdd16b0fcde3e074f1cd8f5f558a08
  Stored in directory: /tmp/pip-ephem-wheel-cache-ccvxx_6c/wheels/46/91/5b/04f12a63a74ba37fbc1e784aa0ede91818e9dbd7fd97630ca3
Successfully built deep-gas-oracle
  Attempting uninstall: deep-gas-oracle
    Found existing installation: deep-gas-oracle 0.1
    Uninstalling deep-gas-oracle-0.1:
      Successfully uninstalled deep-gas-oracle-0.1


## Configs

In [2]:
configs = {
    "SELECTED_MODEL": "exp_1",
    "SELECTED_WEIGHTS": "epoch_best.pth",
    "URGENCY_PARAMETER": 1.0,
    "PERCENTILE_VALUE": 20, # 20th percentile as in the paper
    "BIG_QUERY_CREDENTIALS_PATH": "../credentials/gcp_credentials.json",
    # Dates used for the recommendation evaluation on eth blocks
    "START_DATE": "2019-11-19 00:00:00",
    "END_DATE": "2019-11-24 00:00:00",
    "BLOCK_START": 8965759, # Optional
    "BLOCK_END": 8995344, # Optional
    
    # MAKE SURE TO USE THE SAME VALUES HERE THAN ON THE TRAINED MODEL
    # TODO: Serialize those configs from the model class itself
    "FEATURES": ["mean_gas_price", "min_gas_price", "max_gas_price", "mean_gas_price_24h_lagged", "eth_usd_price"],
    "TARGETS": ["min_gas_price"],
    "WINDOW_SIZE": 200,
    "PREDICT_SIZE": 36, ## 3h forecast length
    "HIDDEN_SIZE": 16,
    "NUM_LAYERS": 1,
    "BATCH_SIZE": 1000,
    "SMOOTH_FRACTION": 0.3, # A lower value of smooth_fraction will result in a smoother curve
    "PATHS": {
        "TRAIN_DATASET_PATH": "../outputs/train_normalized.parquet",
        "TEST_DATASET_PATH": "../outputs/test_normalized.parquet",
        "SCALER_PATH": "../outputs/scaler.pickle",
        "RUNS_DIR_PATH": "../outputs/runs",
    },
    "DEVICE": "cpu"
}

### Model Loading

In [3]:
PATH = Path(configs["PATHS"]["RUNS_DIR_PATH"]) / configs["SELECTED_MODEL"] / "weights" / configs["SELECTED_WEIGHTS"]

model = GruMultiStep(features=configs["FEATURES"],
                     targets=configs["TARGETS"],
                     input_length=configs["WINDOW_SIZE"],
                     output_length=configs["PREDICT_SIZE"],
                     hidden_size=configs["HIDDEN_SIZE"],
                     num_layers=configs["NUM_LAYERS"],
                     device=configs["DEVICE"],
                     logs_dir=configs["PATHS"]["RUNS_DIR_PATH"])

model.load_state_dict(torch.load(PATH))
model

Parameters (param name -> param count):
	gru.weight_ih_l0 -> 240
	gru.weight_hh_l0 -> 768
	gru.bias_ih_l0 -> 48
	gru.bias_hh_l0 -> 48
	linear_head.weight -> 576
	linear_head.bias -> 36
Total param count: 1716


GruMultiStep(
  (gru): GRU(5, 16, batch_first=True)
  (linear_head): Linear(in_features=16, out_features=36, bias=True)
  (loss_func): MSELoss()
)

In [4]:
train_dataset = pd.read_parquet(configs["PATHS"]["TRAIN_DATASET_PATH"])
test_dataset = pd.read_parquet(configs["PATHS"]["TEST_DATASET_PATH"])
scaler = Normalizer.load_from_file(configs["PATHS"]["SCALER_PATH"])

test_dataset.head()

,mean_gas_price,min_gas_price,max_gas_price,mean_gas_price_24h_lagged,eth_usd_price
timestamp,,,,,
2019-11-19 00:05:00,-0.720053,-0.543821,-0.788088,-0.655300,0.990053
2019-11-19 00:10:00,-0.707747,-0.398689,-0.732144,-0.464569,0.984021
2019-11-19 00:15:00,-0.635097,-0.493932,-0.571965,-0.452274,0.996296
2019-11-19 00:20:00,-0.699065,-0.665194,-0.878513,-0.647407,0.980847
2019-11-19 00:25:00,-0.512848,-0.489645,-0.587268,-0.406151,0.978519


## Deep gas oracle initialisation

In [5]:
oracle = EthGasPriceOracle(model=model,
                           training_normalized_dataframe=train_dataset,
                           scaler=scaler,
                           percentile_value=configs["PERCENTILE_VALUE"])

Slopes fitting done from training set...
min slope: -0.019485336225908572 / max slop: 0.01176639490418735


In [6]:

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = configs["BIG_QUERY_CREDENTIALS_PATH"]

In [7]:

client = bigquery.Client()

In [8]:

df_blocks = fetch_eth_blocks(client,
                             start_date=configs["START_DATE"],
                             end_date=configs["END_DATE"])

In [9]:

if "BLOCK_START" in list(configs.keys()) and configs["BLOCK_START"] is not None:
    block_start = configs["BLOCK_START"]
else:
    block_start = df_blocks[df_blocks.index[0], "number"]
if "BLOCK_END" in list(configs.keys()) and configs["BLOCK_END"] is not None:
    block_end = configs["BLOCK_END"]
else:
    block_end = df_blocks[df_blocks.index[-1], "number"]

In [10]:
df_blocks = df_blocks[(df_blocks["number"] >= block_start) & (df_blocks["number"] <= block_end)].copy()

df_blocks.head()

,number,transaction_count,gas_limit,gas_used_percentage,mean_gas_price,min_gas_price,max_gas_price
timestamp,,,,,,,
2019-11-20 01:56:12,8965759,229,99.54717,99.33974,17.537735,2.000000,60.000000
2019-11-20 01:56:24,8965760,175,99.44997,99.38679,11.582892,1.010402,343.597384
2019-11-20 01:56:32,8965761,160,99.49845,99.40293,3.952118,1.000000,26.000000
2019-11-20 01:56:51,8965762,100,99.40161,37.55431,18.159403,10.000000,60.000000
2019-11-20 01:57:09,8965763,208,99.35956,98.93110,10.469883,1.000000,163.208757


## Deep gas oracle evaluation

In [11]:
df_recommendation = oracle.recommend(test_dataset, configs["URGENCY_PARAMETER"])
df_recommendation.head()

,gas_price_recommendation
2019-11-19 16:40:00,2.084048
2019-11-19 16:45:00,2.021903
2019-11-19 16:50:00,2.229775
2019-11-19 16:55:00,2.391255
2019-11-19 17:00:00,2.291546


In [12]:
results = evaluate_recommender(df_blocks, df_recommendation)

results[["blocks_waited", "price_recommended", "inclusion_price"]].describe()

,blocks_waited,price_recommended,inclusion_price
count,28596.000000,28596.000000,28596.000000
mean,3.807316,2.325371,1.095438
std,8.255545,0.666092,0.419654
min,0.000000,0.000000,0.000000
25%,1.000000,1.804821,1.000000
50%,2.000000,2.398061,1.000000
75%,3.000000,2.723975,1.000000
max,139.000000,4.435253,4.000000


## Geth baseline evaluation

In [13]:
df_recommendation = geth_recommend(df_blocks)
df_recommendation.head()

,gas_price_recommendation
timestamp,
2019-11-20 02:19:46,2.0000
2019-11-20 02:19:51,1.6712
2019-11-20 02:20:05,1.6712
2019-11-20 02:20:11,1.6712
2019-11-20 02:20:22,1.6712


In [14]:
results = evaluate_recommender(df_blocks, df_recommendation)
results[["blocks_waited", "price_recommended", "inclusion_price"]].describe()

,blocks_waited,price_recommended,inclusion_price
count,28496.000000,28496.000000,28496.000000
mean,1.949782,4.787156,2.613649
std,1.883193,5.177903,3.829732
min,0.000000,0.000000,0.000000
25%,1.000000,1.200000,1.000000
50%,1.000000,3.000000,1.000000
75%,2.000000,6.000000,2.000000
max,22.000000,30.000000,30.000000


We can see that the proposed deep gas oracle got better recommendations than geth, as found in the paper.
Changing the Urgency parameter allow for faster and more expensive transaction or slower and cheaper ones.
(I did not perform hyperparameters optimization)